In [ ]:
import sys

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets, models,transforms as T
import matplotlib.pyplot as plt
from torchinfo import summary
import numpy as np
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import DataLoader
from datetime import datetime

print(sys.path)

from torchvision import datasets, models,transforms as T
from torch.utils.data import DataLoader
import torch

base_path = "/home/ec2-user/SageMaker/data/64/rice-diseases-image-dataset/Transformed-RiceDiseaseDataset"
num_workers=10

transform = T.Compose(
    [
        T.ToTensor(),
    ]
)

def get_training_data_loder(batch_size):
    training_data = datasets.ImageFolder(base_path + '/train', transform=transform)
    kwargs = {'num_workers': num_workers, 'pin_memory': True}
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, **kwargs)
    return train_dataloader

def get_test_data_loader(batch_size):
    training_data = datasets.ImageFolder(base_path + '/validation', transform=transform)
    test_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    return test_dataloader

net =  models.densenet201()

if torch.cuda.is_available():
    print(f"CUDA is available. Devices: {torch.cuda.device_count()}")
    print(f"Current Device:{torch.cuda.current_device()}")
    torch.cuda.device(0)
    print(f"Current Device:{torch.cuda.current_device()}")
    print("Moving Model to CUDA.")
    net.cuda()

    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    
summary(net, input_size=(1,3,64,64))

learning_rate = 1
batch_size = 32
epochs = 100

train_dataloader = get_training_data_loder(batch_size)
test_dataloader = get_test_data_loader(batch_size)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    print(datetime.now())
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        #print(batch, ":before:", datetime.now())
        #X = X.cuda()
        #X = image_transforms(X)
        #print(X)
        X = X.cuda()
        y = y.cuda()
        pred = model(X)
        #print(batch, ":after:", datetime.now())
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"batch: {batch}, loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.cuda()
            y = y.cuda()
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [ ]:
for i in range(0,100):
    train_loop(train_dataloader,net,loss_fn,optimizer)
    test_loop(test_dataloader,net,loss_fn)

In [ ]:
!pip install torchinfo